# Visualizing the data
In this notebook we will creae several visualization based on the curated metadata. <br>
The goal is to start providing context to the data base and investigate the distribution of fermented foods and thier microbiome. <br>
The data will not reflect true abundance of microbes in different foods/areas since it is based on presensce/absence counts from the metadata table. <br>

In [206]:
# make sure all libraries are installed before running this code

# importing standard libraries
import pandas as pd
import numpy as np
import time
from IPython.display import display
import json

# importing plotly libraries for dinamic visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.subplots as sp
import plotly.offline as pyo
import plotly.tools as tls
import plotly

# importing folium for better visualization using maps
import folium

# importing geolocation libraries so we can get the coordinates of the countries
import pycountry
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.extra.rate_limiter import RateLimiter



In [155]:
# loading the dataset
dataset = pd.read_csv('data/Food_MAGs_curated_metadata_250421_corrected_merged_final_v2.csv')

display(dataset.head(), dataset.shape)

,mag_id,sample_description,fermented_food,specific_substrate,substrate_category,general_category,completeness,contamination,contigs,total_length,...,main_ferment,acid_type,acid_level,alcohol_lovel,protein_degradation,fat_degradation,added_ingredients,fermentation_temp,aging_time,agin_temp
0,C-03.Ssa-BR,raw canastra cheese,cheese,milk,dairy,fermented_dairy,97.97,1.10,182,1896140,...,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
1,C-R02.bin.1,raw canastra cheese,cheese,milk,dairy,fermented_dairy,92.75,1.03,84,3174852,...,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
2,C-R03.bin.7,raw canastra cheese,cheese,milk,dairy,fermented_dairy,98.91,0.00,161,2047554,...,acid; amino acids,lactic,medium,none,high,medium,salt; rennet,mesophilic,>1 month,cold
3,C-R06.bin.10,raw araxa cheese,cheese,milk,dairy,fermented_dairy,91.62,1.35,147,3636195,...,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold
4,C-R06.bin.12,raw araxa cheese,cheese,milk,dairy,fermented_dairy,100.00,0.64,54,3940686,...,acid; amino acids,lactic,medium,none,medium,low,salt; rennet,mesophilic,>1 month,cold


(13858, 45)

In [ ]:
# correcting several naming/spelling mistales I founf in the dataset

# change aland islands fo Finland in country
dataset['country'] = dataset['country'].replace({'Aland Islands': 'Finland'})
# change boliva to Bolivia in country
dataset['country'] = dataset['country'].replace({'Boliva': 'Bolivia'})
# change Belgium:Antwerp to Belgium
dataset['country'] = dataset['country'].replace({'Belgium:Antwerp': 'Belgium'})

# correct 'cheese ' to 'cheese'
dataset['level3_food_name'] = dataset['level3_food_name'].replace({'cheese ': 'cheese'})

# correct choclate to chocolate
dataset['level3_food_name'] = dataset['level3_food_name'].replace({'choclate': 'chocolate'})

# correct saerkraut to sauerkraut
dataset['level3_food_name'] = dataset['level3_food_name'].replace({'saerkraut': 'sauerkraut'})



In [211]:
dataset.level3_food_name.nunique(),dataset.level3_food_name.unique()

(192,
 array(['cheese canastra', 'cheese araxa', 'cheese serro', 'ambriss',
        'labneh', 'cheese serdaleh', 'miso yellow pea', 'miso',
        'vinegar apple cider', 'cider apple', 'pickle Chinese traditional',
        'beer', 'black cherry', 'blueberry', 'cabbage',
        'cassava sour starch', 'wine chardonnay', 'cheese cheddar',
        'cheese ', 'cheese kopanisti', 'chicha', 'pickle chinese',
        'pickle chinese traditional', 'chrysanthemum', 'choclate',
        'pickle cucumber', 'cheese toma', 'coffee', 'tofu stinky',
        'molasses', 'cassava', 'milk fermented', 'fish fermented',
        'pickle tomato', 'pickle peper', 'pickle radish',
        'soybean meal fermented', 'miang', 'pickle mixed vegetables',
        'vinegar broth', 'sourdough', 'fig fermented', 'fu-tsai',
        'gochujang', 'cider grape', 'jeotgal', 'tibicos', 'kefir grain',
        'kimchi', 'kimcki', 'kimchi scallion', 'kimchi wakame', 'kombucha',
        'moromi', 'makgeolli', 'sausage', 'meju',

In [210]:
pd.set_option('display.max_rows', None)
display(food_counts.groupby('food').Count.sum().sort_index())

food
agadagidi                     2
agousha                       1
amazake                       1
ambriss                       1
attieke                       1
bean paste                    2
beer                          7
beer ginger                   2
boza                          1
brukina                       1
burukutu                      3
carrot sticks                 1
cheese                        3
cheese                     1425
cheese araxa                  3
cheese blue                   1
cheese brie                   2
cheese brine                 51
cheese canastra               3
cheese cheddar                2
cheese continental pink       2
cheese parmesan               1
cheese pink                   1
cheese russian                1
cheese ruz'a                  1
cheese serdaleh               2
cheese serro                  2
cheese tomme                  5
cheese wagashi                1
cheese wara                   3
cheese, curd                  1
che

In [156]:
# visualizing the number of samples (sample_accession) per country
# Get unique sample_accession and country pairs
unique_samples_by_country = dataset[['sample_accession', 'country']].drop_duplicates()

# # Count samples per country and sort in descending order
country_counts = unique_samples_by_country.value_counts('country').sort_values(ascending=False)
country_counts = country_counts.reset_index()
country_counts.columns = ['country', 'count']
# remove values that are lists
country_counts = country_counts[~country_counts['country'].str.contains('\[', na=False)]

# Create horizontal bar plot
fig = px.bar(country_counts, 
             y='country', 
             x='count',
             orientation='h',
             title="Number of unique samples per country",
             category_orders={'country': country_counts['country'].tolist()},
             height=800,
             width=800,)  

fig.update_layout(xaxis_title="Number of unique samples",
                 yaxis_title="Country",
                 yaxis=dict(tickfont=dict(size=8)))  # reduced font size for y-axis labels

fig.update_traces(marker_color='blue', marker_line_color='black',
                  marker_line_width=1.5, opacity=0.6)
fig.show()

In [157]:
# Define continent mapping
continent_map = {
    'Italy': 'Europe', 'Ireland': 'Europe', 'Spain': 'Europe', 'Austria': 'Europe', 
    'USA': 'North America', 'United Kingdom': 'Europe', 'Canada': 'North America',
    'Genrmany': 'Europe', 'France': 'Europe', 'China': 'Asia', 'Nigeria': 'Africa',
    'Australia': 'Oceania', 'Mexico': 'North America', 'Belgium': 'Europe',
    'Taiwan': 'Asia', 'India': 'Asia', 'Brazil': 'South America', 'Ghana': 'Africa',
    'Denmark': 'Europe', 'Singapore': 'Asia', 'Greece': 'Europe', 'Malaysia': 'Asia',
    'Norway': 'Europe', 'Turkey': 'Asia', 'Saudi Arabia': 'Asia', 
    'South Africa': 'Africa', 'Bulgaria': 'Europe', 'Russia': 'Europe',
    'Indonesia': 'Asia', 'Ecuador': 'South America', 'New Zealand': 'Oceania',
    'Finland': 'Europe', 'Korea': 'Asia', 'Colombia': 'South America',
    'Hong Kong': 'Asia', 'Estonia': 'Europe', 'hilippines': 'Asia',
    'Burkina Faso': 'Africa', 'Sweden': 'Europe', 'Tunisia': 'Africa',
    'Aland Islands': 'Europe', 'Croatia': 'Europe', 'Israel': 'Asia',
    'Lebanon': 'Asia', 'Portugal': 'Europe', 'Kenya': 'Africa',
    'Boliva': 'South America', 'Switzerland': 'Europe', 'Ethiopia': 'Africa',
    'Benin': 'Africa', 'Japan': 'Asia', 'Germany': 'Europe', 'Thailand': 'Asia',
    'Myanmar': 'Asia', 'Korean': 'Asia', 'Turkey ': 'Asia', 'Belgium:Antwerp': 'Europe'
}

# Add continent column
country_counts['continent'] = country_counts['country'].map(continent_map)

# Create scatter_geo plot with color by continent
fig = px.scatter_geo(country_counts,
                    locations='country',
                    locationmode='country names',
                    size='count',
                    color='continent',
                    hover_name='country',
                    hover_data={'count': True, 'continent': True},
                    projection='natural earth',
                    title='Number of Samples per Country')

fig.update_traces(marker=dict(opacity=0.6, line=dict(width=1, color='black')))

fig.update_layout(
    title_x=0.5,
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
        # Add zoom capabilities
        scope='world',
        showland=True,
        showcountries=True,
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
        # Enable zoom
        visible=True
    ),
    width=1000,
    height=600
)

# Add buttons for zooming to different regions
fig.update_layout(
    updatemenus=[{
        'buttons': [
            {'args': [{'geo.scope': 'world'}], 'label': 'World', 'method': 'relayout'},
            {'args': [{'geo.scope': 'europe'}], 'label': 'Europe', 'method': 'relayout'},
            {'args': [{'geo.scope': 'asia'}], 'label': 'Asia', 'method': 'relayout'},
            {'args': [{'geo.scope': 'africa'}], 'label': 'Africa', 'method': 'relayout'},
            {'args': [{'geo.scope': 'north america'}], 'label': 'North America', 'method': 'relayout'},
            {'args': [{'geo.scope': 'south america'}], 'label': 'South America', 'method': 'relayout'}
        ],
        'direction': 'down',
        'showactive': True,
        'x': 1.2,
        'y': 0.6  # Moved down from 0.9 to 0.75 to place below typical legend position
    }]
)

fig.show()

In [191]:
# visualizing the number of samples (sample_accession) per country
# Get unique sample_accession and country pairs
unique_food_by_country = dataset[['sample_accession', 'country','level3_food_name']].drop_duplicates()

# Count samples per country and sort in descending order
food_counts = unique_food_by_country.groupby(['country', 'level3_food_name']).size().reset_index(name='Count')
food_counts = food_counts.sort_values(['country','Count'], ascending=[False, True])


In [192]:
# correcting for rows that have multiple countries in the same cell
# First, create a copy of the dataframe
new_food_counts = food_counts.copy()

# Find rows where country contains '[' 
mask = new_food_counts['country'].str.contains('\[', na=False)
rows_to_expand = new_food_counts[mask].copy()

# Remove these rows from the original dataframe
new_food_counts = new_food_counts[~mask]

# Process rows that need to be expanded
for idx, row in rows_to_expand.iterrows():
    # Extract country names from the string
    countries = eval(row['country'])
    
    # Create new rows for each country
    for country in countries:
        new_row = pd.DataFrame({
            'country': [country],
            'level3_food_name': [row['level3_food_name']],
            'Count': [row['Count']]
        })
        new_food_counts = pd.concat([new_food_counts, new_row], ignore_index=True)

# Sort the resulting dataframe
new_food_counts = new_food_counts.sort_values(['country', 'Count'], ascending=[True, False])

# Update food_counts with the new dataframe
food_counts = new_food_counts.copy()

In [ ]:
 # change korean to Korea
food_counts['country'] = food_counts['country'].replace({'korean': 'Korea'})

In [194]:
# add a column with the % of a food items per country
food_counts['percentage'] = food_counts.groupby('country')['Count'].transform(lambda x: x / x.sum() * 100).round(2)



In [195]:
# change the column name level3_food_name to food
food_counts = food_counts.rename(columns={'level3_food_name': 'food'})


In [203]:
pd.set_option('display.max_rows', None)
display(food_counts.groupby('food').Count.sum().sort_values(ascending=False))

food
cheese                     1425
tibicos fig                 377
kefir                       270
salami                       73
whey                         71
sourdough                    69
cheese brine                 51
sausage                      44
choclate                     35
koumiss                      33
tea fermented                28
wine                         22
yogurt                       21
nunu                         17
sausage italian              14
probiotics                   12
sauerkraut                   12
soy sauce                    11
kimchi                       10
tibicos                       9
mustard leaf                  9
ogi                           8
miso                          7
beer                          7
coffee                        7
chilli sauce                  6
kinema                        6
iru                           6
kunu                          6
cheese tomme                  5
hongeo                        5
pul

In [205]:
food_counts.Count.sum()

np.int64(2859)

In [174]:
fig1 = px.bar(food_counts,
             y='country',
             x='percentage',
             color='food',  # Use food column for color
             orientation='h',
             title="Percentage of researched food per country",
             category_orders={'country': food_counts['country'].tolist()},
             height=1000,
             width=800)  # Increased width to accommodate legend

# Update layout
fig1.update_layout(
    showlegend=True,
    legend_title_text="Food Types",
    # Adjust legend position and size
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    # Adjust margins to prevent label cutoff
    margin=dict(r=200)  # Increased right margin for legend
)

fig1.show()

In [175]:
import pycountry
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
from geopy.extra.rate_limiter import RateLimiter

# Initialize geolocator with increased timeout
geolocator = Nominatim(user_agent="country_locator", timeout=10)

# Create a rate-limited version of the geocoding function
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to get coordinates with retry mechanism
def get_coordinates(country_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Manual mapping for special cases
            country_mapping = {
                'USA': 'United States',
                'Korea': 'South Korea',
                'hilippines': 'Philippines',
                'Genrmany': 'Germany'
            }
            
            search_name = country_mapping.get(country_name, country_name)
            
            try:
                # Try using pycountry first
                country = pycountry.countries.lookup(search_name)
                search_name = country.name
            except (LookupError, AttributeError):
                # If pycountry fails, use the original or mapped name
                pass
            
            # Add explicit country type to search
            location = geocode(f"{search_name}", featuretype='country')
            if location:
                return pd.Series([location.latitude, location.longitude])
            
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            if attempt == max_retries - 1:
                print(f"Failed to geocode {country_name} after {max_retries} attempts: {str(e)}")
            else:
                time.sleep(2 ** attempt)
                continue
        except Exception as e:
            print(f"Error processing {country_name}: {str(e)}")
            break
            
    return pd.Series([None, None])

# Process unique countries
unique_countries = food_counts['country'].unique()
coordinates_df = pd.DataFrame(unique_countries, columns=['country'])
coordinates_df[['latitude', 'longitude']] = coordinates_df['country'].apply(get_coordinates)

# Merge coordinates back to the main DataFrame
food_counts = food_counts.merge(coordinates_df, on='country', how='left')


In [176]:
food_counts[food_counts['latitude'].isna()]

,country,food,count,percentage,latitude,longitude
76,Hong Kong,tibicos fig,5,100.0,NaN,NaN
210,Taiwan,kefir,28,100.0,NaN,NaN


In [177]:
# find the lat and lon for Taiwan
taiwan = geocode('Taiwan')
# add the lat and lon for Taiwan to the food_counts dataframe
food_counts.loc[food_counts['country'] == 'Taiwan', 'latitude'] = taiwan.latitude
food_counts.loc[food_counts['country'] == 'Taiwan', 'longitude'] = taiwan.longitude

#find the lat and lon for hong kong
hong_kong = geocode('Hong Kong')
# add the lat and lon for Hong Kong to the food_counts dataframe
food_counts.loc[food_counts['country'] == 'Hong Kong', 'latitude'] = hong_kong.latitude
food_counts.loc[food_counts['country'] == 'Hong Kong', 'longitude'] = hong_kong.longitude

In [188]:
import folium
import plotly.express as px
from IPython.display import display
import json

# Create the base map with a lighter color scheme
m = folium.Map(location=[20, 0], zoom_start=2, tiles='CartoDB positron')

for country in food_counts['country'].unique():
    country_data = food_counts[food_counts['country'] == country]
    lat = country_data['latitude'].values[0]
    lon = country_data['longitude'].values[0]
    
    if pd.notnull(lat) and pd.notnull(lon):
        # Create pie chart using plotly
        fig = px.pie(country_data, 
                     values='percentage',
                     names='food',
                     title=country)
        
        # Update layout with larger dimensions and better visibility
        fig.update_layout(
            width=300,
            height=300,
            margin=dict(l=20, r=20, t=40, b=20),
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=-0.3,
                xanchor="center",
                x=0.5
            )
        )
        
        # Convert plotly figure to HTML with increased size
        html_str = fig.to_html(
            include_plotlyjs='cdn',
            full_html=False,
            config={'displayModeBar': False}
        )
        
        # Create larger iframe and popup
        iframe = folium.IFrame(html=html_str, width=350, height=350)
        popup = folium.Popup(iframe, max_width=400)
        
        # Add a more visible marker
        folium.CircleMarker(
            location=[lat, lon],
            radius=8,
            color='blue',
            fill=True,
            popup=popup
        ).add_to(m)

# Display the map
display(m)